<a href="https://colab.research.google.com/github/seyonechithrananda/ncov-ligand-protein/blob/master/Cross_Fold_Broad_Scaffold_GAT_ncov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Now that we've ran all of our benchmarks and found the GAT-based classificaiton model trained on a Scaffold split to be the strongest, lets combine our SARS-COV-2 + SARS-COV-1 datasets (multi-task classifier) and train so we can utilize the model to screen repurposing drug assays for inhibition. We'll use a 90-10 train-test split so we can use as much data as possible for training the model (here, the test split only serves the purpose of verifing that the model performs alright).

In [2]:
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y -c conda-forge rdkit
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')


--2020-05-08 16:14:17--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h7b6447c_0
    - conda==4.8.2=py37_0
    - cryptography==2.8=py37h1ba5d50_0
    - idna==2.8=py37_0
    - ld_impl_linux-64==2.33.1=h53a641e_7
    - libedit==3.1.2

In [3]:
!conda install -c dglteam dgl-cuda10.1

Solving environment: \ | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - dgl-cuda10.1


The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            conda-forge::certifi-2020.4.5.1-py37h~ --> pkgs/main::certifi-2020.4.5.1-py37_0


Proceed ([y]/n)? y

Preparing transaction: / done
Verifying transaction: \ done
Executing transaction: / done


In [5]:
!conda install -c dglteam dgllife


Solving environment: - \ | / - \ | / - \ | / done

# All requested packages already installed.



In [6]:
!conda install pandas

Solving environment: | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pandas


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2020.4.5~ --> pkgs/main::ca-certificates-2020.1.1-0
  conda              conda-forge::conda-4.8.3-py37hc8dfbb8~ --> pkgs/main::conda-4.8.3-py37_0
  openssl            conda-forge::openssl-1.1.1g-h516909a_0 --> pkgs/main::openssl-1.1.1g-h7b6447c_0


Proceed ([y]/n)? y

Preparing transaction: / done
Verifying transaction: \ | done
Executing transaction: - done


In [0]:
import os 
import sys 
import pandas as pd

# train --> balanced dataset
dataset_file = '/content/drive/My Drive/Project De Novo/AID1706_binarized_sars_mpro_xchem_full_eval_actives_12k_samples.csv - AID1706_binarized_sars.csv'
dataset = pd.read_csv(dataset_file)


In [5]:
dataset.head

<bound method NDFrame.head of                                                   smiles  activity
0      C1CC(C1)C(=O)NC2=CC=C(C=C2)N(C(C3=CC(=CC=C3)F)...         1
1      CC(C)(C)C1=CC=C(C=C1)N(C(C2=CN=NC=C2)C(=O)NC(C...         1
2      CC(C)(C)NC(=O)C(C1=CSC=C1)N(C2=CC=C(C=C2)N)C(=...         1
3      CC(C)C(=O)NC1=CC=C(C=C1)N(C(C2=CSC=C2)C(=O)NC(...         1
4      CC(C)C(=O)NC1=CC=C(C=C1)N(CC2=CSC=C2)C(=O)CN3C...         1
...                                                  ...       ...
12874                   CC(C)C=1C=CC(NC(=O)N2CCOCC2)=CC1         0
12875                        CN(CC(=O)O)C(=O)C=1C=CC=CN1         0
12876                  CN1CCN(CC1)C(=O)C=2C=CC(F)=C(F)C2         0
12877                      FC=1C=CC=C(F)C1C(=O)N2CCCCCC2         0
12878                             FC=1C=CC=NC1NCC2CCOCC2         0

[12879 rows x 2 columns]>

In [6]:
dataset.tail

<bound method NDFrame.tail of                                                   smiles  activity
0      C1CC(C1)C(=O)NC2=CC=C(C=C2)N(C(C3=CC(=CC=C3)F)...         1
1      CC(C)(C)C1=CC=C(C=C1)N(C(C2=CN=NC=C2)C(=O)NC(C...         1
2      CC(C)(C)NC(=O)C(C1=CSC=C1)N(C2=CC=C(C=C2)N)C(=...         1
3      CC(C)C(=O)NC1=CC=C(C=C1)N(C(C2=CSC=C2)C(=O)NC(...         1
4      CC(C)C(=O)NC1=CC=C(C=C1)N(CC2=CSC=C2)C(=O)CN3C...         1
...                                                  ...       ...
12874                   CC(C)C=1C=CC(NC(=O)N2CCOCC2)=CC1         0
12875                        CN(CC(=O)O)C(=O)C=1C=CC=CN1         0
12876                  CN1CCN(CC1)C(=O)C=2C=CC(F)=C(F)C2         0
12877                      FC=1C=CC=C(F)C1C(=O)N2CCCCCC2         0
12878                             FC=1C=CC=NC1NCC2CCOCC2         0

[12879 rows x 2 columns]>

In [7]:
from dgllife.data import MoleculeCSVDataset
from dgllife.data.csv_dataset import *
from dgllife.utils.featurizers import *
from dgllife.utils.mol_to_graph import *

# featurize bigraph/molecular graph set for train (SARS-COV-1) set
csv_dataset = MoleculeCSVDataset(dataset, smiles_to_graph=smiles_to_bigraph, node_featurizer=CanonicalAtomFeaturizer(),
                               edge_featurizer=CanonicalBondFeaturizer(), smiles_column='smiles', cache_file_path='/content/drive/My Drive/Project De Novo/graph_featurizer/sars-mpro-xchem.bin', task_names=['activity'])

Loading previously saved dgl graphs...


In [12]:
from google.colab import files
files.upload()

Saving utils.py to utils (1).py


{'utils.py': b'import dgl\nimport numpy as np\nimport random\nimport torch\n\nfrom dgllife.utils.featurizers import one_hot_encoding\nfrom dgllife.utils.splitters import RandomSplitter\n\ndef set_random_seed(seed=0):\n    """Set random seed.\n    Parameters\n    ----------\n    seed : int\n        Random seed to use\n    """\n    random.seed(seed)\n    np.random.seed(seed)\n    torch.manual_seed(seed)\n    if torch.cuda.is_available():\n        torch.cuda.manual_seed(seed)\n\n\ndef load_dataset_for_classification(args):\n    """Load dataset for classification tasks.\n    Parameters\n    ----------\n    args : dict\n        Configurations.\n    Returns\n    -------\n    dataset\n        The whole dataset.\n    train_set\n        Subset for training.\n    val_set\n        Subset for validation.\n    test_set\n        Subset for test.\n    """\n    assert args[\'dataset\'] in [\'Tox21\']\n    if args[\'dataset\'] == \'Tox21\':\n        from dgllife.data import Tox21\n        dataset = Tox

In [0]:
args = {
    'random_seed': 2,
    'batch_size': 128,
    'lr': 1e-3,
    'num_epochs': 250,
    'node_data_field': 'h',
    'frac_train': 0.8,
    'frac_val': 0.1,
    'frac_test': 0.1,
    'in_feats': 74,
    'gat_hidden_feats': [32, 32],
    'classifier_hidden_feats': 64,
    'num_heads': [4, 4],
    'patience': 50,
    'smiles_to_graph': smiles_to_bigraph,
    'node_featurizer': CanonicalAtomFeaturizer(),
    'metric_name': 'roc_auc_score',
    'model': 'GAT'
}


In [0]:
import numpy as np
import torch
import utils

from dgllife.model import load_pretrained
from dgllife.utils import EarlyStopping, Meter
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam
from torch.utils.data import DataLoader


from utils import set_random_seed, load_dataset_for_classification, collate_molgraphs, load_model

from dgllife.model import GATPredictor

args['device'] = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
set_random_seed(args['random_seed'])



In [31]:
from dgllife.utils.splitters import ScaffoldSplitter

train_scaffold = ScaffoldSplitter.k_fold_split(csv_dataset, k=5)

Start initializing RDKit molecule instances...
Creating RDKit molecule instance 1000/12879
Creating RDKit molecule instance 2000/12879
Creating RDKit molecule instance 3000/12879
Creating RDKit molecule instance 4000/12879
Creating RDKit molecule instance 5000/12879
Creating RDKit molecule instance 6000/12879
Creating RDKit molecule instance 7000/12879
Creating RDKit molecule instance 8000/12879
Creating RDKit molecule instance 9000/12879
Creating RDKit molecule instance 10000/12879
Creating RDKit molecule instance 11000/12879
Creating RDKit molecule instance 12000/12879
Start computing Bemis-Murcko scaffolds.
Computing Bemis-Murcko for compound 1000/12879
Computing Bemis-Murcko for compound 2000/12879
Computing Bemis-Murcko for compound 3000/12879
Computing Bemis-Murcko for compound 4000/12879
Computing Bemis-Murcko for compound 5000/12879
Computing Bemis-Murcko for compound 6000/12879
Computing Bemis-Murcko for compound 7000/12879
Computing Bemis-Murcko for compound 8000/12879
Comput

In [32]:
print (len(csv_dataset))

print(len(train_scaffold[0][0]))
print(len(train_scaffold[0][1]))
print(len(train_scaffold[1][0]))
print(len(train_scaffold[1][1]))
print(len(train_scaffold[2][0]))
print(len(train_scaffold[2][1]))
print(len(train_scaffold[3][0]))
print(len(train_scaffold[3][1]))
print(len(train_scaffold[4][0]))
print(len(train_scaffold[4][1]))

print(csv_dataset[1])

12879
10303
2576
10303
2576
10303
2576
10303
2576
10304
2575
('CC(C)(C)C1=CC=C(C=C1)N(C(C2=CN=NC=C2)C(=O)NC(C)(C)C)C(=O)C3=CC=CO3', DGLGraph(num_nodes=32, num_edges=68,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)}), tensor([1.]), tensor([1.]))


In [15]:
print(type(csv_dataset))
print(type(train_scaffold[0][0]))

<class 'dgllife.data.csv_dataset.MoleculeCSVDataset'>
<class 'dgl.data.utils.Subset'>


In [0]:
train_loader_1 = DataLoader(train_scaffold[0][0],  batch_size=args['batch_size'],
                              collate_fn=collate_molgraphs)
val_loader_1 = DataLoader(train_scaffold[0][1],  batch_size=args['batch_size'],
                              collate_fn=collate_molgraphs)

train_loader_2 = DataLoader(train_scaffold[1][0],  batch_size=args['batch_size'],
                              collate_fn=collate_molgraphs)
val_loader_2 = DataLoader(train_scaffold[1][1],  batch_size=args['batch_size'],
                              collate_fn=collate_molgraphs)

train_loader_3 = DataLoader(train_scaffold[2][0],  batch_size=args['batch_size'],
                              collate_fn=collate_molgraphs)
val_loader_3 = DataLoader(train_scaffold[2][1],  batch_size=args['batch_size'],
                              collate_fn=collate_molgraphs)

train_loader_4 = DataLoader(train_scaffold[3][0],  batch_size=args['batch_size'],
                              collate_fn=collate_molgraphs)
val_loader_4 = DataLoader(train_scaffold[3][1],  batch_size=args['batch_size'],
                              collate_fn=collate_molgraphs)

train_loader_5 = DataLoader(train_scaffold[4][0],  batch_size=args['batch_size'],
                              collate_fn=collate_molgraphs)
val_loader_5 = DataLoader(train_scaffold[4][1],  batch_size=args['batch_size'],
                              collate_fn=collate_molgraphs)


In [18]:
print(len(train_loader_1))
print(len(train_loader_2))
print(len(val_loader_1))
print(len(val_loader_5))

81
81
21
21


In [0]:
args['n_tasks'] = 1

model_1 = GATPredictor(in_feats=args['node_featurizer'].feat_size('h'),
                             hidden_feats=args['gat_hidden_feats'],
                             num_heads=args['num_heads'],
                             classifier_hidden_feats=args['classifier_hidden_feats'],
                             n_tasks=args['n_tasks'])

model_2 = GATPredictor(in_feats=args['node_featurizer'].feat_size('h'),
                             hidden_feats=args['gat_hidden_feats'],
                             num_heads=args['num_heads'],
                             classifier_hidden_feats=args['classifier_hidden_feats'],
                             n_tasks=args['n_tasks'])

model_3 = GATPredictor(in_feats=args['node_featurizer'].feat_size('h'),
                             hidden_feats=args['gat_hidden_feats'],
                             num_heads=args['num_heads'],
                             classifier_hidden_feats=args['classifier_hidden_feats'],
                             n_tasks=args['n_tasks'])

model_4 = GATPredictor(in_feats=args['node_featurizer'].feat_size('h'),
                             hidden_feats=args['gat_hidden_feats'],
                             num_heads=args['num_heads'],
                             classifier_hidden_feats=args['classifier_hidden_feats'],
                             n_tasks=args['n_tasks'])

model_5 = GATPredictor(in_feats=args['node_featurizer'].feat_size('h'),
                             hidden_feats=args['gat_hidden_feats'],
                             num_heads=args['num_heads'],
                             classifier_hidden_feats=args['classifier_hidden_feats'],
                             n_tasks=args['n_tasks'])

stopper_1 = EarlyStopping(patience=args['patience'], mode='higher', filename='/content/drive/My Drive/Project De Novo/GAT/model_1_train_scaffold_combined_data_250.pth')
stopper_2 = EarlyStopping(patience=args['patience'], mode='higher', filename='/content/drive/My Drive/Project De Novo/GAT/model_2_train_scaffold_combined_data_250.pth')
stopper_3 = EarlyStopping(patience=args['patience'], mode='higher', filename='/content/drive/My Drive/Project De Novo/GAT/model_3_train_scaffold_combined_data_250.pth')
stopper_4 = EarlyStopping(patience=args['patience'], mode='higher', filename='/content/drive/My Drive/Project De Novo/GAT/model_4_train_scaffold_combined_data_250.pth')
stopper_5 = EarlyStopping(patience=args['patience'], mode='higher', filename='/content/drive/My Drive/Project De Novo/GAT/model_5_train_scaffold_combined_data_250.pth')

optimizer_1 = Adam(model_1.parameters(), lr=args['lr'])
optimizer_2 = Adam(model_2.parameters(), lr=args['lr'])
optimizer_3 = Adam(model_3.parameters(), lr=args['lr'])
optimizer_4 = Adam(model_4.parameters(), lr=args['lr'])
optimizer_5 = Adam(model_5.parameters(), lr=args['lr'])


In [0]:

import dgl.backend as F



train_num_pos = F.sum(csv_dataset.labels, dim=0)
train_num_indices = F.sum(csv_dataset.mask, dim=0)
train_task_pos_weights = (train_num_indices - train_num_pos) / train_num_pos

loss_criterion = BCEWithLogitsLoss(pos_weight=train_task_pos_weights.to(args['device']),
                                    reduction='none')



In [23]:
model_1.to(args['device'])
model_2.to(args['device'])
model_3.to(args['device'])
model_4.to(args['device'])
model_5.to(args['device'])


GATPredictor(
  (gnn): GAT(
    (gnn_layers): ModuleList(
      (0): GATLayer(
        (gat_conv): GATConv(
          (fc): Linear(in_features=74, out_features=128, bias=False)
          (feat_drop): Dropout(p=0.0, inplace=False)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (leaky_relu): LeakyReLU(negative_slope=0.2)
          (res_fc): Linear(in_features=74, out_features=128, bias=False)
        )
      )
      (1): GATLayer(
        (gat_conv): GATConv(
          (fc): Linear(in_features=128, out_features=128, bias=False)
          (feat_drop): Dropout(p=0.0, inplace=False)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (leaky_relu): LeakyReLU(negative_slope=0.2)
          (res_fc): Linear(in_features=128, out_features=128, bias=False)
        )
      )
    )
  )
  (readout): WeightedSumAndMax(
    (weight_and_sum): WeightAndSum(
      (atom_weighting): Sequential(
        (0): Linear(in_features=32, out_features=1, bias=True)
        (1): Sigmoid()


In [24]:
print(train_task_pos_weights)

tensor([23.5782])


In [0]:
def predict(args, model, bg):
    node_feats = bg.ndata.pop(args['node_data_field']).to(args['device'])
    if args.get('edge_featurizer', None) is not None:
        edge_feats = bg.edata.pop(args['edge_data_field']).to(args['device'])
        return model(bg, node_feats, edge_feats)
    else:
        return model(bg, node_feats)

def run_a_train_epoch(args, epoch, model, data_loader, loss_criterion, optimizer):
    model.train()
    train_meter = Meter()
    for batch_id, batch_data in enumerate(data_loader):
        smiles, bg, labels, masks = batch_data
        labels, masks = labels.to(args['device']), masks.to(args['device'])
        print(bg)
        logits = predict(args, model, bg)
        # Mask non-existing labels
        loss = (loss_criterion(logits, labels) * (masks != 0).float()).mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print('epoch {:d}/{:d}, batch {:d}/{:d}, loss {:.4f}'.format(
            epoch + 1, args['num_epochs'], batch_id + 1, len(data_loader), loss.item()))
        train_meter.update(logits, labels, masks)
    train_score = np.mean(train_meter.compute_metric(args['metric_name']))
    print('epoch {:d}/{:d}, training {} {:.4f}'.format(
        epoch + 1, args['num_epochs'], args['metric_name'], train_score))

def run_an_eval_epoch(args, model, data_loader):
    model.eval()
    eval_meter = Meter()
    with torch.no_grad():
        for batch_id, batch_data in enumerate(data_loader):
            smiles, bg, labels, masks = batch_data
            labels = labels.to(args['device'])
            logits = predict(args, model, bg)
            eval_meter.update(logits, labels, masks)
    return np.mean(eval_meter.compute_metric(args['metric_name']))


In [26]:
# model #1 --> cross validation
for epoch in range(args['num_epochs']):
        # Train
        run_a_train_epoch(args, epoch, model_1, train_loader_1, loss_criterion, optimizer_1)

        # Validation and early stop
        val_score = run_an_eval_epoch(args, model_1, val_loader_1)
        early_stop = stopper_1.step(val_score, model_1)
        print('epoch {:d}/{:d}, validation {} {:.4f}, best validation {} {:.4f}'.format(
            epoch + 1, args['num_epochs'], args['metric_name'],
            val_score, args['metric_name'], stopper_1.best_score))
        if early_stop:
            break

DGLGraph(num_nodes=2540, num_edges=5328,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})


/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


Streaming output truncated to the last 5000 lines.
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 116/250, batch 59/81, loss 0.0669
DGLGraph(num_nodes=3198, num_edges=7000,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 116/250, batch 60/81, loss 0.1291
DGLGraph(num_nodes=2684, num_edges=5522,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 116/250, batch 61/81, loss 0.4082
DGLGraph(num_nodes=2589, num_edges=5410,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 116/250, batch 62/81, loss 0.1015
DGLGraph(num_nodes=2598, num_edges=5442,
         ndata_schemes={'h': Scheme(shape=(74,), dtyp

In [27]:
# model #2 --> cross validation
for epoch in range(args['num_epochs']):
        # Train
        run_a_train_epoch(args, epoch, model_2, train_loader_2, loss_criterion, optimizer_2)

        # Validation and early stop
        val_score = run_an_eval_epoch(args, model_2, val_loader_2)
        early_stop = stopper_2.step(val_score, model_2)
        print('epoch {:d}/{:d}, validation {} {:.4f}, best validation {} {:.4f}'.format(
            epoch + 1, args['num_epochs'], args['metric_name'],
            val_score, args['metric_name'], stopper_2.best_score))
        if early_stop:
            break

Streaming output truncated to the last 5000 lines.
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 93/250, batch 59/81, loss 0.1006
DGLGraph(num_nodes=3198, num_edges=7000,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 93/250, batch 60/81, loss 0.3332
DGLGraph(num_nodes=2684, num_edges=5522,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 93/250, batch 61/81, loss 0.8892
DGLGraph(num_nodes=2589, num_edges=5410,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 93/250, batch 62/81, loss 0.1743
DGLGraph(num_nodes=2598, num_edges=5442,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=to

In [28]:
# model #3 --> cross validation
for epoch in range(args['num_epochs']):
        # Train
        run_a_train_epoch(args, epoch, model_3, train_loader_3, loss_criterion, optimizer_3)

        # Validation and early stop
        val_score = run_an_eval_epoch(args, model_3, val_loader_3)
        early_stop = stopper_3.step(val_score, model_3)
        print('epoch {:d}/{:d}, validation {} {:.4f}, best validation {} {:.4f}'.format(
            epoch + 1, args['num_epochs'], args['metric_name'],
            val_score, args['metric_name'], stopper_3.best_score))
        if early_stop:
            break

Streaming output truncated to the last 5000 lines.
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 45/250, batch 59/81, loss 0.3500
DGLGraph(num_nodes=3198, num_edges=7000,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 45/250, batch 60/81, loss 0.5442
DGLGraph(num_nodes=2684, num_edges=5522,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 45/250, batch 61/81, loss 2.0941
DGLGraph(num_nodes=2589, num_edges=5410,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 45/250, batch 62/81, loss 0.3192
DGLGraph(num_nodes=2598, num_edges=5442,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=to

In [29]:
# model #4 --> cross validation
for epoch in range(args['num_epochs']):
        # Train
        run_a_train_epoch(args, epoch, model_4, train_loader_4, loss_criterion, optimizer_4)

        # Validation and early stop
        val_score = run_an_eval_epoch(args, model_4, val_loader_4)
        early_stop = stopper_4.step(val_score, model_4)
        print('epoch {:d}/{:d}, validation {} {:.4f}, best validation {} {:.4f}'.format(
            epoch + 1, args['num_epochs'], args['metric_name'],
            val_score, args['metric_name'], stopper_4.best_score))
        if early_stop:
            break

Streaming output truncated to the last 5000 lines.
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 42/250, batch 59/81, loss 0.3367
DGLGraph(num_nodes=3170, num_edges=6922,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 42/250, batch 60/81, loss 0.4647
DGLGraph(num_nodes=2657, num_edges=5464,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 42/250, batch 61/81, loss 1.3789
DGLGraph(num_nodes=2589, num_edges=5410,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 42/250, batch 62/81, loss 0.3707
DGLGraph(num_nodes=2598, num_edges=5442,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=to

In [30]:
# model #5 --> cross validation
for epoch in range(args['num_epochs']):
        # Train
        run_a_train_epoch(args, epoch, model_5, train_loader_5, loss_criterion, optimizer_5)

        # Validation and early stop
        val_score = run_an_eval_epoch(args, model_5, val_loader_5)
        early_stop = stopper_5.step(val_score, model_5)
        print('epoch {:d}/{:d}, validation {} {:.4f}, best validation {} {:.4f}'.format(
            epoch + 1, args['num_epochs'], args['metric_name'],
            val_score, args['metric_name'], stopper_5.best_score))
        if early_stop:
            break

Streaming output truncated to the last 5000 lines.
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 44/250, batch 59/81, loss 0.2955
DGLGraph(num_nodes=3170, num_edges=6922,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 44/250, batch 60/81, loss 0.4076
DGLGraph(num_nodes=2322, num_edges=4866,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 44/250, batch 61/81, loss 0.6193
DGLGraph(num_nodes=2577, num_edges=5396,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 44/250, batch 62/81, loss 0.4933
DGLGraph(num_nodes=2788, num_edges=5874,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=to

In [12]:
broad_file = '/content/drive/My Drive/Project De Novo/broad_repurposing_library.csv'

broad = pd.read_csv(broad_file)
broad.head

<bound method NDFrame.head of                                                  smiles
0     CO\N=C(\C(=O)NC1C2SCC(CSc3nnnn3C)=C(N2C1=O)C(O...
1           CN1CCN(CC1)c1c(F)cc2c3c1SCCn3cc(C(O)=O)c2=O
2     C[C@H]1CN(C[C@@H](C)N1)c1c(F)c(N)c2c(c1F)n(cc(...
3     CCN1CCN(C(=O)N[C@@H](C(=O)N[C@H]2[C@H]3SCC(CSc...
4             CCn1cc(C(O)=O)c(=O)c2cc(F)c(nc12)N1CCNCC1
...                                                 ...
6106                          C(Oc1ccccc1C1CC1)C1=NCCN1
6107                  CCOc1ccc(NC(C)=Nc2ccc(OCC)cc2)cc1
6108    CC[C@@H]1CN2C(=N1)c1[nH]c(nc1N(C)C2=O)-c1ccccc1
6109                            N=C(NNc1ccccc1)c1ccccc1
6110             CS(=O)(=O)Nc1c(O)ccc2C(CCCc12)C1=NCCN1

[6111 rows x 1 columns]>

In [13]:
from dgllife.utils import smiles_to_bigraph

smiles_list = broad['smiles']

print(len(smiles_list))
print(smiles_list[5000])

6111
CCNC1=N[C@H]2[C@H](O[C@H](CO)[C@@H](O)[C@@H]2O)S1


In [24]:

broad_graph = []

for i, smiles in enumerate(smiles_list):
  broad_graph.append(smiles_to_bigraph(smiles, node_featurizer=CanonicalAtomFeaturizer()))

print(len(broad_graph))
print(type(broad_graph))


6111
<class 'list'>


In [25]:
# verify SMILES are in right order in both smiles_list and graph list
print(smiles_to_bigraph(smiles_list[2000], node_featurizer=CanonicalAtomFeaturizer()))
print(broad_graph[2000])

DGLGraph(num_nodes=15, num_edges=32,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={})
DGLGraph(num_nodes=15, num_edges=32,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={})


In [57]:
args['n_tasks']=1
gat_1 = GATPredictor(in_feats=args['node_featurizer'].feat_size('h'),
                             hidden_feats=args['gat_hidden_feats'],
                             num_heads=args['num_heads'],
                             classifier_hidden_feats=args['classifier_hidden_feats'],
                             n_tasks=args['n_tasks'])

gat_1.load_state_dict(torch.load('/content/drive/My Drive/Project De Novo/GAT/model_1_train_scaffold_combined_data_250.pth')['model_state_dict'])

gat_2 = GATPredictor(in_feats=args['node_featurizer'].feat_size('h'),
                             hidden_feats=args['gat_hidden_feats'],
                             num_heads=args['num_heads'],
                             classifier_hidden_feats=args['classifier_hidden_feats'],
                             n_tasks=args['n_tasks'])

gat_2.load_state_dict(torch.load('/content/drive/My Drive/Project De Novo/GAT/model_2_train_scaffold_combined_data_250.pth')['model_state_dict'])

gat_3 = GATPredictor(in_feats=args['node_featurizer'].feat_size('h'),
                             hidden_feats=args['gat_hidden_feats'],
                             num_heads=args['num_heads'],
                             classifier_hidden_feats=args['classifier_hidden_feats'],
                             n_tasks=args['n_tasks'])

gat_3.load_state_dict(torch.load('/content/drive/My Drive/Project De Novo/GAT/model_3_train_scaffold_combined_data_250.pth')['model_state_dict'])

gat_4 = GATPredictor(in_feats=args['node_featurizer'].feat_size('h'),
                             hidden_feats=args['gat_hidden_feats'],
                             num_heads=args['num_heads'],
                             classifier_hidden_feats=args['classifier_hidden_feats'],
                             n_tasks=args['n_tasks'])

gat_4.load_state_dict(torch.load('/content/drive/My Drive/Project De Novo/GAT/model_4_train_scaffold_combined_data_250.pth')['model_state_dict'])

gat_5 = GATPredictor(in_feats=args['node_featurizer'].feat_size('h'),
                             hidden_feats=args['gat_hidden_feats'],
                             num_heads=args['num_heads'],
                             classifier_hidden_feats=args['classifier_hidden_feats'],
                             n_tasks=args['n_tasks'])

gat_5.load_state_dict(torch.load('/content/drive/My Drive/Project De Novo/GAT/model_5_train_scaffold_combined_data_250.pth')['model_state_dict'])



<All keys matched successfully>

In [58]:
gat_1.to(args['device'])
gat_2.to(args['device'])
gat_3.to(args['device'])
gat_4.to(args['device'])
gat_5.to(args['device'])


GATPredictor(
  (gnn): GAT(
    (gnn_layers): ModuleList(
      (0): GATLayer(
        (gat_conv): GATConv(
          (fc): Linear(in_features=74, out_features=128, bias=False)
          (feat_drop): Dropout(p=0.0, inplace=False)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (leaky_relu): LeakyReLU(negative_slope=0.2)
          (res_fc): Linear(in_features=74, out_features=128, bias=False)
        )
      )
      (1): GATLayer(
        (gat_conv): GATConv(
          (fc): Linear(in_features=128, out_features=128, bias=False)
          (feat_drop): Dropout(p=0.0, inplace=False)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (leaky_relu): LeakyReLU(negative_slope=0.2)
          (res_fc): Linear(in_features=128, out_features=128, bias=False)
        )
      )
    )
  )
  (readout): WeightedSumAndMax(
    (weight_and_sum): WeightAndSum(
      (atom_weighting): Sequential(
        (0): Linear(in_features=32, out_features=1, bias=True)
        (1): Sigmoid()


In [60]:
import torch.nn.functional as F

# Getting predictions (activity scores for each SMILES graph)

broad_graph = []
scores = []

for i, smiles in enumerate(smiles_list):
  broad_graph.append(smiles_to_bigraph(smiles, node_featurizer=CanonicalAtomFeaturizer()))

print(len(broad_graph))
print(type(broad_graph))

# verify SMILES are in right order in both smiles_list and graph list
print(smiles_to_bigraph(smiles_list[2000], node_featurizer=CanonicalAtomFeaturizer()))
print(broad_graph[2000])

gat_1.eval()
gat_1.cpu()

gat_2.eval()
gat_2.cpu()

gat_3.eval()
gat_3.cpu()

gat_4.eval()
gat_4.cpu()

gat_5.eval()
gat_5.cpu()



for graph in broad_graph:
  node_feats = graph.ndata.pop(args['node_data_field'])
  #y = predict(args, pretrained_model, graph)
  y_1 = gat_1(graph, node_feats)
  logits_1 = y_1.data.cpu()#.numpy()
  score_np1 = F.sigmoid(logits_1).numpy()

  y_2 = gat_2(graph, node_feats)
  logits_2 = y_2.data.cpu()#.numpy()
  score_np2 = F.sigmoid(logits_2).numpy()

  y_3 = gat_3(graph, node_feats)
  logits_3 = y_3.data.cpu()#.numpy()
  score_np3 = F.sigmoid(logits_3).numpy()

  y_4 = gat_4(graph, node_feats)
  logits_4 = y_4.data.cpu()#.numpy()
  score_np4 = F.sigmoid(logits_4).numpy()

  y_5 = gat_5(graph, node_feats)
  logits_5 = y_5.data.cpu()#.numpy()
  score_np5 = F.sigmoid(logits_5).numpy()


  #print(score_np)
  scores.append((score_np1[0][0]+score_np2[0][0]+score_np3[0][0]+score_np4[0][0]+score_np5[0][0])/5)

print (logits)
print(scores)
'''
  score_1 = predict(args, model, broad_graph)
  score_1 = predict(args, model, broad_graph)
  score_1 = predict(args, model, broad_graph)
  score_1 = predict(args, model, broad_graph)
'''


6111
<class 'list'>
DGLGraph(num_nodes=15, num_edges=32,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={})
DGLGraph(num_nodes=15, num_edges=32,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={})


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([[-0.5893]])
[0.2802781581878662, 0.32343647480010984, 0.24607434272766113, 0.48661341667175295, 0.14736266136169435, 0.2745830535888672, 0.20974531173706054, 0.2796262264251709, 0.31053709983825684, 0.24605603218078614, 0.25430521965026853, 0.2689237356185913, 0.3275089502334595, 0.24628844261169433, 0.27544736862182617, 0.2037100076675415, 0.17473034858703612, 0.22876014709472656, 0.20066194534301757, 0.2137092113494873, 0.3955472469329834, 0.2507993459701538, 0.16934574842453004, 0.3719766616821289, 0.14997975826263427, 0.14055519104003905, 0.3027090072631836, 0.1711361050605774, 0.2986863613128662, 0.21867685317993163, 0.3746514320373535, 0.17281703948974608, 0.30557856559753416, 0.26144311428070066, 0.2513755798339844, 0.3676572322845459, 0.41167259216308594, 0.29116067886352537, 0.2547262668609619, 0.3696759700775146, 0.29395527839660646, 0.15361015796661376, 0.43370866775512695, 0.2859159231185913, 0.2672618865966797, 0.3191995620727539, 0.4739814758300781, 0.289742231369

'\n  score_1 = predict(args, model, broad_graph)\n  score_1 = predict(args, model, broad_graph)\n  score_1 = predict(args, model, broad_graph)\n  score_1 = predict(args, model, broad_graph)\n'

In [68]:
broad_graph = []

for i, smiles in enumerate(smiles_list):
  broad_graph.append(smiles_to_bigraph(smiles, node_featurizer=CanonicalAtomFeaturizer()))

# 9, 60, 30, 71, 72

print(F.sigmoid(gat_1(broad_graph[3], broad_graph[3].ndata.pop(args['node_data_field']))))
print(scores[3])

tensor([[0.0969]], grad_fn=<SigmoidBackward>)
0.48661341667175295


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [72]:
print(len(scores))

broad.insert(column='activity', value=scores, loc=1)

broad.head

6111


<bound method NDFrame.head of                                                  smiles  activity
0     CO\N=C(\C(=O)NC1C2SCC(CSc3nnnn3C)=C(N2C1=O)C(O...  0.280278
1           CN1CCN(CC1)c1c(F)cc2c3c1SCCn3cc(C(O)=O)c2=O  0.323436
2     C[C@H]1CN(C[C@@H](C)N1)c1c(F)c(N)c2c(c1F)n(cc(...  0.246074
3     CCN1CCN(C(=O)N[C@@H](C(=O)N[C@H]2[C@H]3SCC(CSc...  0.486613
4             CCn1cc(C(O)=O)c(=O)c2cc(F)c(nc12)N1CCNCC1  0.147363
...                                                 ...       ...
6106                          C(Oc1ccccc1C1CC1)C1=NCCN1  0.178507
6107                  CCOc1ccc(NC(C)=Nc2ccc(OCC)cc2)cc1  0.260985
6108    CC[C@@H]1CN2C(=N1)c1[nH]c(nc1N(C)C2=O)-c1ccccc1  0.206859
6109                            N=C(NNc1ccccc1)c1ccccc1  0.451018
6110             CS(=O)(=O)Nc1c(O)ccc2C(CCCc12)C1=NCCN1  0.229077

[6111 rows x 2 columns]>

In [0]:
broad.to_csv('/content/drive/My Drive/Project De Novo/GAT/broad_ensemble_predictions.csv')

In [37]:
import torch.nn.functional as F

scores_normalized = []

for s in scores:
  print (s)
  scores_normalized.append(F.sigmoid(s))

print(scores_normalized)

tensor([[0.1299]])
tensor([[-0.4040]])
tensor([[-0.7257]])
tensor([[0.9443]])
tensor([[-0.9985]])
tensor([[-0.2735]])
tensor([[-0.9210]])
tensor([[0.4872]])
tensor([[0.0033]])
tensor([[-0.1556]])
tensor([[-0.4719]])
tensor([[-0.3111]])
tensor([[-0.0097]])
tensor([[-0.3420]])
tensor([[0.2401]])
tensor([[-0.6104]])
tensor([[-0.7947]])
tensor([[-0.4039]])
tensor([[-0.5385]])
tensor([[-0.4178]])
tensor([[0.3612]])
tensor([[-0.1567]])
tensor([[-0.8774]])
tensor([[0.3513]])
tensor([[-0.7615]])
tensor([[-0.6155]])
tensor([[0.2574]])
tensor([[-0.8862]])
tensor([[0.0400]])
tensor([[-0.1587]])
tensor([[0.4798]])
tensor([[-0.8531]])
tensor([[-0.1894]])
tensor([[-0.2877]])
tensor([[-0.7299]])
tensor([[0.4658]])
tensor([[0.4894]])
tensor([[0.3031]])
tensor([[-0.0292]])
tensor([[0.1355]])
tensor([[-0.1201]])
tensor([[-0.5487]])
tensor([[0.6861]])
tensor([[0.0321]])
tensor([[-0.8042]])
tensor([[0.2829]])
tensor([[1.3476]])
tensor([[-0.0684]])
tensor([[0.5748]])
tensor([[-0.0584]])
tensor([[-1.1125]])

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Streaming output truncated to the last 5000 lines.
tensor([[-0.1188]])
tensor([[-0.8621]])
tensor([[-0.3930]])
tensor([[2.6417]])
tensor([[-0.5609]])
tensor([[0.3396]])
tensor([[-0.3565]])
tensor([[0.2737]])
tensor([[0.7803]])
tensor([[0.7431]])
tensor([[-0.9957]])
tensor([[-0.4519]])
tensor([[-0.5784]])
tensor([[0.3903]])
tensor([[0.5139]])
tensor([[1.6429]])
tensor([[1.5672]])
tensor([[-1.2323]])
tensor([[0.2017]])
tensor([[-1.2179]])
tensor([[1.4022]])
tensor([[1.4963]])
tensor([[-0.7227]])
tensor([[0.9225]])
tensor([[-0.6621]])
tensor([[-0.0542]])
tensor([[1.4894]])
tensor([[0.6920]])
tensor([[-0.1056]])
tensor([[0.3020]])
tensor([[0.6402]])
tensor([[-0.1672]])
tensor([[-0.5909]])
tensor([[1.6171]])
tensor([[-1.3214]])
tensor([[1.8784]])
tensor([[0.1016]])
tensor([[-1.0679]])
tensor([[0.2688]])
tensor([[0.5919]])
tensor([[1.2110]])
tensor([[-1.8845]])
tensor([[0.0287]])
tensor([[0.3963]])
tensor([[-0.8124]])
tensor([[0.6698]])
tensor([[-0.7984]])
tensor([[-0.1750]])
tensor([[-1.071

In [38]:
print(arr[0][0])
print (type(arr[0][0]))
broad_graph = []

for i, smiles in enumerate(smiles_list):
  broad_graph.append(smiles_to_bigraph(smiles, node_featurizer=CanonicalAtomFeaturizer()))

# prediction array in correct order
print((predict(args, pretrained_model, broad_graph[3]).data.cpu().numpy()))

0.12992822
<class 'numpy.float32'>
[[0.9442535]]


Now that the score works, here are the action items.

1. Write code to get score for each graph-SMILEs, append to list of scores.
2. Validate that scores are in correct order by getting score for a graph in broad_graph[i] list and comparing it to scores[i]. should be same for predictions done by same model.
3. Add column with all scores attached besides each SMILES string. Save CSV.
4. nice to have: add scores of all 5 models and average to get final score (see commented code above). 

Action item: get top 50 SMILES to pass to ChemProp

In [0]:
# This cell isn't run as there is no test class, in order to save data. best validation is treated as metric. (similar to how k-fold cross validation is done with two tuples)
stopper.load_checkpoint(model)
test_score = run_an_eval_epoch(args, model, test_loader)
print('test {} {:.4f}'.format(args['metric_name'], test_score))
